# PyTorchを利用してMNISTする

▼ 参考記事
[【PyTorch】MNISTのサンプルプログラム](https://imagingsolution.net/deep-learning/pytorch/pytorch_mnist_sample_program/)

In [1]:
import torchvision.transforms as transforms
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Hyper Parameters
num_epochs = 10
num_batch = 100
learning_rate = 0.001
image_size = 28 * 28

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform = transforms.Compose([
    transforms.ToTensor()
])

In [3]:
# Preparing training and test data

# Training data
train_data = MNIST(
    '/content/datasets/mnist',
    train=True,
    download=True,
    transform=transform,
)
train_loader = DataLoader(
    train_data,
    batch_size=num_batch,
    shuffle=True,
)

# Test data
test_data = MNIST(
    '/content/datasets/mnist',
    train=False,
    download=False,
    transform=transform,
)
test_loader = DataLoader(
    test_data,
    batch_size=num_batch,
    shuffle=True,
)

100%|██████████| 9912422/9912422 [00:00<00:00, 37348261.52it/s]


Extracting /content/datasets/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /content/datasets/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 72492934.66it/s]

Extracting /content/datasets/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /content/datasets/mnist/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 28752001.35it/s]


Extracting /content/datasets/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/datasets/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12149571.92it/s]

Extracting /content/datasets/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/datasets/mnist/MNIST/raw



In [4]:
class Net(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()

    self.l1 = nn.Linear(input_size, 100) # From a input layer to a hidden layer.
    self.l2 = nn.Linear(100, output_size) # From a hidden layer to a output layer.

  def forward(self, x):
    x = self.l1(x)
    x = torch.sigmoid(x)
    x = self.l2(x)
    return F.log_softmax(x, dim=1)

In [5]:
# Create a neural network.
model = Net(image_size, 10).to(device)

# Loss function
loss_func = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Traning
model.train() # Change mode to training

for epoch in range(num_epochs):
  loss_sum = 0

  for inputs, labels in train_loader:
    # Send data to GPU if it can do
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Initialize optimizer
    optimizer.zero_grad()

    # Perform the neural network.
    inputs = inputs.view(-1, image_size)
    outputs = model(inputs)

    # Calculate loss
    loss = loss_func(outputs, labels)
    loss_sum += loss

    # Calculate gradiation
    loss.backward()

    # Update its weights
    optimizer.step()

  # Show the progress
  print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss_sum.item() / len(train_loader)}')

  # Save its weights
  torch.save(model.state_dict(), 'model_weights.pth')

Epoch: 1/10, Loss: 0.6647649129231771
Epoch: 2/10, Loss: 0.26708773295084637
Epoch: 3/10, Loss: 0.21005208333333333
Epoch: 4/10, Loss: 0.17555706024169923
Epoch: 5/10, Loss: 0.15128894805908202
Epoch: 6/10, Loss: 0.13234212239583334
Epoch: 7/10, Loss: 0.11689402262369791
Epoch: 8/10, Loss: 0.1038913917541504
Epoch: 9/10, Loss: 0.09399867375691731
Epoch: 10/10, Loss: 0.08451857884724935


In [7]:
# Evaluation
model.eval() # Change mode to eval

loss_sum = 0
correct = 0

with torch.no_grad():
  for inputs, labels in test_loader:

    # Send data to GPU if it can do
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Perform the neural network.
    inputs = inputs.view(-1, image_size)
    outputs = model(inputs)

    loss_sum += loss_func(outputs, labels)

    pred = outputs.argmax(1)
    correct += pred.eq(labels.view_as(pred)).sum().item()

  print(f'Loss: {loss_sum.item() / len(test_loader)}, Accuracy: {100 * correct / len(test_data)}% ({correct}/{len(test_data)})')

Loss: 0.10286139488220215, Accuracy: 96.94% (9694/10000)


# ONNXへの変換

上記のモデル作成後に実行してください。

In [8]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 29.8 MB/s eta 0:00:00


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

image_size = 28 * 28
learning_rate = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

# If you separate this code another file on your environment, plasee remove comment out to use Net class.
#
# class Net(nn.Module):
#   def __init__(self, input_size, output_size):
#     super().__init__()

#     self.l1 = nn.Linear(input_size, 100) # From a input layer to a hidden layer.
#     self.l2 = nn.Linear(100, output_size) # From a hidden layer to a output layer.

#   def forward(self, x):
#     x = self.l1(x)
#     x = torch.sigmoid(x)
#     x = self.l2(x)
#     return F.log_softmax(x, dim=1)

# Create a neural network.
model = Net(image_size, 10).to(device)

checkpoint = torch.load('model_weights.pth')
model.load_state_dict(checkpoint)

torch.onnx.export(
    model=model,
    args=torch.randn(1, 784),
    f='model.onnx',
    export_params=True,
    input_names=['input'],
    output_names=['output'],
)